In [22]:
import cv2
import numpy as np

cam = cv2.VideoCapture(0)   #acceccing the default camera

#here we are setting the size of the camera frame
cam.set(3,640)            
cam.set(4,480)            

#for setting the brightness
cam.set(10,150)           


markerColors= [[0,153,0,4,255,255]]

tipColors= [[17,125,249]]

Points= []                              #[x, y, color index]  checking the value of x and y and drawing the points on 
    

In [23]:
def findContours(img) :
    
    contours, Hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    x, y, width, height = 0,0,0,0     #if the object if not detected the value returned will be zero 
    
    for cnt in contours :
        area = cv2.contourArea(cnt)   #getting the contour Area
        
        
        if area>500 :
    #cv2.drawContours(ResultImg, cnt, -1, (255,0,0),3)      #this function just helps in creating a bounding box over the object
            peri = cv2.arcLength(cnt, True)
            
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            
            
            x, y, width, height = cv2.boundingRect(approx)
            
    return x+width//2,y           #returning the value for returning the tip's center position of the marker to write

In [24]:
def findingColors(img, markerColors, tipColors) :
    
    HSVimg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    newPoints=[]
    
    i = 0                               #counter to change the color tipColors values
    
    for colors in markerColors : 
        lower = np.array(colors[0:3])
        upper = np.array(colors[3:6])
    
        mask = cv2.inRange(HSVimg, lower, upper)
    
        x, y = findContours(mask)     #getting the value to draw our tip
        cv2.circle(ResultImg,(x,y),8,tipColors[i],cv2.FILLED)        #creating the tip of our marker
        
        if x!=0 and y!=0 :
            newPoints.append([x, y, i])
        
        ++i
        
    return newPoints

In [25]:
def screenDrawing(Points, tipColors) :
    
    for pt in Points :
        cv2.circle(ResultImg,(pt[0],pt[1]),8,tipColors[pt[2]],cv2.FILLED)
    

In [26]:
while True :
    success, img = cam.read()  #it will save our images in the 'img'  variable, and 
                               #'success' variable will store a boolean varible it will tell weather theimage is captured or not
    ResultImg = img.copy()
        
    newPoints= findingColors(img, markerColors, tipColors)
    if len(newPoints) != 0 :
        for nwp in newPoints :
            Points.append(nwp)
        
    if len(Points)!=0 :
        screenDrawing(Points, tipColors)
        
    
    cv2.imshow("camera image",ResultImg)    #this will show the result
    
    if cv2.waitKey(1) & 0xFF ==ord('q') :
        break;

cam.release()
cv2.destroyAllWindows()